<h1>Compute training data</h1>

In [11]:
# import pandas as pd
# from flag_pennants.compute import runAnalysis

# dtype_dict = {
#     'Open': 'float64',
#     'High': 'float64',
#     'Low': 'float64',
#     'Close': 'float64',
#     'Volume': 'float64'
# }

# # Define columns to be parsed as dates
# date_columns = ['Time']

# # Read the CSV file with specified column data types and date parsing
# history = pd.read_csv("./Data/History/XAUUSD_M5_2023_formatted.csv", dtype=dtype_dict, parse_dates=date_columns)

# _,_,_,_,_ = runAnalysis(history['Close'].to_numpy(), print_results=False)

<h1>Import computed datas</h1>

In [12]:
import pandas as pd

concat_bear_flag_df_full = pd.read_csv('./Data/results/bear_flag_df.csv')
concat_bear_pennant_df_full = pd.read_csv('./Data/results/bear_pennant_df.csv')
concat_bull_pennant_df_full = pd.read_csv('./Data/results/bull_pennant_df.csv')
concat_bull_flag_df_full = pd.read_csv('./Data/results/bull_flag_df.csv')

<h1>Shape data for training</h1>

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

concat_bear_pennant_df_full = concat_bear_pennant_df_full.rename(columns={'pennant_width' : 'flag_width', 'pennant_height' : 'flag_height'})
concat_bull_pennant_df_full = concat_bull_pennant_df_full.rename(columns={'pennant_width' : 'flag_width', 'pennant_height' : 'flag_height'})

bull_flag = pd.concat([concat_bull_pennant_df_full, concat_bull_flag_df_full], ignore_index=True)
bear_flag = pd.concat([concat_bear_pennant_df_full, concat_bear_flag_df_full], ignore_index=True)


print((bull_flag['flag_width'] + bull_flag['pole_width']).mean())
print((bull_flag['flag_width'] + bull_flag['pole_width']).min())
print((bull_flag['flag_width'] + bull_flag['pole_width']).max())

# X_bull = bull_flag.drop(columns=['return', , 'Unnamed: 0', 'id','base_x'])
X_bull = bull_flag[['flag_width','flag_height','pole_width','pole_height','support_slope','resist_slope','support_intercept','resist_intercept']]
Y_bull = bull_flag[['return']]
X_bear = bear_flag[['flag_width','flag_height','pole_width','pole_height','support_slope','resist_slope','support_intercept','resist_intercept']]
Y_bear = bear_flag[['return']]

sc = MinMaxScaler(feature_range=(0,1))
X_bull = sc.fit_transform(X_bull)
Y_bull = sc.fit_transform(Y_bull)
X_bear = sc.fit_transform(X_bear)
Y_bear = sc.fit_transform(Y_bear)

split_ratio = 0.2


X_train_bull, X_test_bull, Y_train_bull, Y_test_bull = train_test_split(X_bull, Y_bull, test_size=split_ratio, random_state=42)
X_train_bear, X_test_bear, Y_train_bear, Y_test_bear = train_test_split(X_bear, Y_bear, test_size=split_ratio, random_state=42)

X_train_bull = X_train_bull.reshape((X_train_bull.shape[0], 1, X_train_bull.shape[1]))
Y_train_bull = Y_train_bull.reshape((Y_train_bull.shape[0], 1, Y_train_bull.shape[1]))
X_test_bull = X_test_bull.reshape((X_test_bull.shape[0], 1, X_test_bull.shape[1]))


print(X_train_bull.shape)
print(Y_train_bull.shape)
print(X_test_bull.shape)
print(Y_test_bull.shape)


75.74613686534217
20.0
216.0
(724, 1, 8)
(724, 1, 1)
(182, 1, 8)
(182, 1)


<h1>Build & Train model</h1>
1 - To estimate max win

In [44]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed

# import tensorflow as tf
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate, Flatten
import numpy as np
#tf.random.set_seed(20)
np.random.seed(10)

lstm_input = Input(shape=(1,8), name='lstm_input')
flattened_input = Flatten()(lstm_input)  # Flatten the input to (batch_size, 8)
inputs = Dense(64, name='first_layer')(flattened_input)
inputs = Dense(64, name='dense_layer')(inputs)
inputs = Dense(1, name='output_layer')(inputs)
output = Activation('linear', name='output')(inputs)
model = Model(inputs=lstm_input, outputs=output)
adam = optimizers.Adam()
model.compile(optimizer=adam, loss='mse')
model.fit(x=X_train_bull, y=Y_train_bull, batch_size=20, epochs=20, shuffle=True, validation_split = 0.1)

Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0441 - val_loss: 0.0267
Epoch 2/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0276 - val_loss: 0.0313
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0258 - val_loss: 0.0338
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0237 - val_loss: 0.0263
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0270 - val_loss: 0.0375
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0296 - val_loss: 0.0389
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0290 - val_loss: 0.0268
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0267 - val_loss: 0.0258
Epoch 9/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0255 - val_loss: 0.0269
Epoch 10/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0254 - val_loss: 0.0263
Epoch 11/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0267 - val_loss: 0.0262
Epoch 12/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0282 - val_lo

In [45]:
y_pred = model.predict(X_test_bull)
# print(Y_test_bull)
print(y_pred.shape)

sign_prediction = ((y_pred * Y_test_bull) > 0).sum() / Y_test_bull.shape[0] * 100
with np.errstate(divide='ignore', invalid='ignore'):
    accuracy = np.nanmean(np.where(Y_test_bull != 0, abs(y_pred - Y_test_bull) / Y_test_bull, np.nan))

print(f'Sign predction accuracy: {sign_prediction:.2f} %')
print(f'Accuracy: {accuracy:.2f} %')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
(182, 1)
Sign predction accuracy: 97.25 %
Accuracy: 0.87 %


2 - To estimate trade duration that maximize gain